In [3]:
############ IMPORTS ####################
import sys
sys.path.append("../")
import os
import numpy as np
from scipy import io
from sklearn.decomposition import PCA
from PIL import Image
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, cohen_kappa_score
import torch
from operator import truediv
import record
import keras.backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.losses import categorical_crossentropy
from keras.layers import Dense, Flatten, GRU
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import to_categorical as keras_to_categorical
import tensorflow as tf


############# CONFIGS ##########################

# os.environ["CUDA_VISIBLE_DEVICES"]="2"
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
datasetNames = ["Trento"]
patchsize1 = 11
patchsize2 = 11
batchsize = 64
EPOCH = 200
LR = 0.001

    
def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc*100, average_acc*100

def get_model_compiled(feat_size,seq, Classes):
    func = GRU
    clf = Sequential()
    clf.add(func(64, return_sequences=True, input_shape=(feat_size, seq)))
    clf.add(func(64, return_sequences=True))
    clf.add(Flatten())
    clf.add(Dense(Classes, activation='softmax'))
    clf.compile(loss=categorical_crossentropy, optimizer=Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False), metrics=['accuracy'])
    return clf


for datasetName in datasetNames:
    print(datasetName)
    HSIOnly = True
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    for i in range(1):
        print("----------------------------------Training for ",datasetName,"---------------------------------------------")
    
    try:
        os.makedirs(datasetName)
    except FileExistsError:
        pass
    # Train data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Tr.mat')
    TrainPatch = HSI['Data']
    TrainPatch = TrainPatch.astype(np.float32)
    NC = TrainPatch.shape[3] # NC is number of bands
    
    label = io.loadmat('./../'+datasetName+'11x11/TrLabel.mat')
    TrLabel = label['Data']
    
    # Test data
    HSI = io.loadmat('./../'+datasetName+'11x11/HSI_Te.mat')
    TestPatch = HSI['Data']
    TestPatch = TestPatch.astype(np.float32)
    
    label = io.loadmat('./../'+datasetName+'11x11/TeLabel.mat')
    TsLabel = label['Data']

    
    TrainPatch1 = torch.from_numpy(TrainPatch)
#         TrainPatch1 = TrainPatch1.permute(0,3,1,2)
    TrainLabel1 = torch.from_numpy(TrLabel)-1
    TrainLabel1 = TrainLabel1.long()
    

    TestPatch1 = torch.from_numpy(TestPatch)
#         TestPatch1 = TestPatch1.permute(0,3,1,2)
    TestLabel1 = torch.from_numpy(TsLabel)-1
    TestLabel1 = TestLabel1.long()

    Classes = len(np.unique(TrainLabel1))
    print("Train data shape = ", TrainPatch1.shape)
    print("Train label shape = ", TrainLabel1.shape)
    print("Test data shape = ", TestPatch1.shape)
    print("Test label shape = ", TestLabel1.shape)
    
    KAPPA = []
    OA = []
    AA = []
    ELEMENT_ACC = np.zeros((3, Classes))

    tf.compat.v1.keras.backend.clear_session()
    config = tf.compat.v1.ConfigProto( device_count = {'GPU': 2} ) 
    sess = tf.compat.v1.Session(config=config) 
    tf.compat.v1.keras.backend.set_session(sess)
    g = tf.Graph()
    with g.as_default():
        for iter in range(3):
            n_bands, sequences = TrainPatch1.reshape(TrainPatch1.size(0),TrainPatch1.size(1),-1).cpu().detach().numpy().shape[1:]
            clf = get_model_compiled(n_bands, sequences, Classes)
            valdata = (TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1],-1).cpu().detach().numpy(), keras_to_categorical(TestLabel1.reshape(-1).cpu().detach().numpy(),Classes))
            clf.fit(TrainPatch1.reshape(TrainPatch1.shape[0],TrainPatch1.shape[1],-1).cpu().detach().numpy(), keras_to_categorical(TrainLabel1.reshape(-1).cpu().detach().numpy(),Classes),
                                batch_size=batchsize,
                                epochs=EPOCH,
                                verbose=True,
                                validation_data=valdata,
                                callbacks = [ModelCheckpoint(datasetName+"/best_model_HSIOnly.h5", monitor='val_accuracy', verbose=0, save_best_only=True)])



            clf = load_model(datasetName+"/best_model_HSIOnly.h5")
            pred_y = np.argmax(clf.predict(TestPatch1.reshape(TestPatch1.shape[0],TestPatch1.shape[1],-1).cpu().detach().numpy()), axis=1)

            y_test = TestLabel1.reshape(-1).cpu().detach().numpy()
            oa = accuracy_score(y_test, pred_y)*100
            confusion = confusion_matrix(y_test, pred_y)
            each_acc, aa = AA_andEachClassAccuracy(confusion)
            kappa = cohen_kappa_score(y_test, pred_y)*100
            KAPPA.append(kappa)
            OA.append(oa)
            AA.append(aa)
            ELEMENT_ACC[iter, :] = each_acc
    print("--------" + datasetName + " Training Finished-----------")
    record.record_output(OA, AA, KAPPA, ELEMENT_ACC,'./' + datasetName +'/GRU_Report_' + datasetName +'.txt')

    

Trento
----------------------------------Training for  Trento ---------------------------------------------
Train data shape =  torch.Size([819, 11, 11, 63])
Train label shape =  torch.Size([1, 819])
Test data shape =  torch.Size([29395, 11, 11, 63])
Test label shape =  torch.Size([1, 29395])


2021-10-30 08:48:22.934138: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0035:03:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.50GiB deviceMemoryBandwidth: 836.37GiB/s
2021-10-30 08:48:22.934276: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-10-30 08:48:22.934308: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-10-30 08:48:22.934335: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-10-30 08:48:22.934361: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2021-10-30 08:48:22.934424: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolv

Train on 819 samples, validate on 29395 samples


2021-10-30 08:48:24.379139: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1564] Found device 0 with properties: 
pciBusID: 0035:03:00.0 name: Tesla V100-SXM2-32GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 31.50GiB deviceMemoryBandwidth: 836.37GiB/s
2021-10-30 08:48:24.379208: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.2
2021-10-30 08:48:24.379229: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-10-30 08:48:24.379245: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-10-30 08:48:24.379262: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2021-10-30 08:48:24.379322: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolv

Epoch 1/200
819/819 [==============================] - 4s 5ms/sample - loss: 1.4796 - accuracy: 0.3419 - val_loss: 1.1510 - val_accuracy: 0.8605
Epoch 2/200
819/819 [==============================] - 4s 5ms/sample - loss: 0.9144 - accuracy: 0.6825 - val_loss: 0.8291 - val_accuracy: 0.8097
Epoch 3/200
819/819 [==============================] - 4s 5ms/sample - loss: 0.5754 - accuracy: 0.8877 - val_loss: 0.4920 - val_accuracy: 0.8526
Epoch 4/200
819/819 [==============================] - 4s 5ms/sample - loss: 0.3399 - accuracy: 0.9121 - val_loss: 0.3124 - val_accuracy: 0.9137
Epoch 5/200
819/819 [==============================] - 4s 5ms/sample - loss: 0.2164 - accuracy: 0.9316 - val_loss: 0.3126 - val_accuracy: 0.8847
Epoch 6/200
819/819 [==============================] - ETA: 0s - loss: 0.2250 - accuracy: 0.9170

KeyboardInterrupt: 